<img src='https://user-images.githubusercontent.com/16665629/36117115-329c6220-1041-11e8-98d3-7cd1ce05503c.jpeg'
style="float:right;margin:0 10px 0px 0;width: 150px;">

# Compare observed and theoretical statistical distributions

In [ ]:
sky = 'point' # For the example recipes in this project options are ['point', 'extended', 'hdr']

models_compare = {
    '{}_skymodel1.txt'.format(sky): 'meerkat_{}_clean_mask-cube-image-pybdsf.lsm.html'.format(sky),
    '{}_skymodel2.txt'.format(sky): 'meerkat_{}_ddfacet_mask-cube-image-pybdsf.lsm.html'.format(sky),
    '{}_skymodel3.txt'.format(sky): 'meerkat_{}_lwimager_mask-cube-image-pybdsf.lsm.html'.format(sky),
    '{}_skymodel4.txt'.format(sky): 'meerkat_{}_tclean_mask-cube-image-pybdsf.lsm.html'.format(sky),
    '{}_skymodel5.txt'.format(sky): 'meerkat_{}_wsclean_mask-cube-image-pybdsf.lsm.html'.format(sky),
}# Copy the appropriate model into 'output' then rename 1-5

models_dr = {
    'meerkat_{}_clean_mask-cube-image-pybdsf.lsm.html'.format(sky): 'meerkat_{}_clean_mask-cube-residual.fits'.format(sky),
    'meerkat_{}_ddfacet_mask-cube-image-pybdsf.lsm.html'.format(sky): 'meerkat_{}_ddfacet_mask.cube.app.residual.fits'.format(sky),
    'meerkat_{}_lwimager_mask-cube-image-pybdsf.lsm.html'.format(sky): 'meerkat_{}_lwimager_mask-cube-residual.fits'.format(sky),
    'meerkat_{}_tclean_mask-cube-image-pybdsf.lsm.html'.format(sky): 'meerkat_{}_tclean_mask-cube-residual.fits'.format(sky),
    'meerkat_{}_wsclean_mask-cube-image-pybdsf.lsm.html'.format(sky): 'meerkat_{}_wsclean_mask-cube-residual.fits'.format(sky),
}

RESIDUALS = {
    'meerkat_{}_clean_mask-cube-residual.fits'.format(sky): 'casa_noise.dirty.image.fits',
    'meerkat_{}_ddfacet_mask.cube.app.residual.fits'.format(sky):'ddfacet_noise.cube.app.restored.fits',
    'meerkat_{}_lwimager_mask-cube-residual.fits'.format(sky): 'lwimager_noise.dirty.fits',
    'meerkat_{}_tclean_mask-cube-residual.fits'.format(sky): 'tclean_noise.dirty.image.fits',
    'meerkat_{}_wsclean_mask-cube-residual.fits'.format(sky):'wsclean_noise-cube-dirty.fits',
}

# tolerance for cross-matching the source
if sky is 'point':
    tolerance = 0.00001 # point
else:
    tolerance = 0.0001 # extended

In [ ]:
from ipywidgets import widgets, interact
from IPython.display import Javascript, display
from IPython.display import HTML

def run_all(ev):
    # Runn all cells
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+24)'))

directory = widgets.Text(
    value='output',
    placeholder='output',
    description='work directory:',
    disabled=False
)

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)
display(directory)

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} $( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import glob
import aimfast
import matplotlib
import numpy as np
from plotly import tools
from tabletext import to_text
import plotly.graph_objs as go
from plotly import offline as py
from collections import OrderedDict
from scipy.stats import norm, linregress
from sklearn.metrics import r2_score, mean_squared_error
from plotly.graph_objs import Layout, Figure, XAxis, YAxis
from aimfast.aimfast import residual_image_stats, image_dynamic_range, model_dynamic_range
py.init_notebook_mode(connected=True)

## Photometric Measurements

In [ ]:
flux = aimfast.aimfast.plot_photometry(models_compare, all_sources=True, dir=directory.value,
                                       label=['clean', 'ddfacet', 'lwimager', 'tclean', 'wsclean'])



## Astrometric Measurements

In [ ]:
position = aimfast.aimfast.plot_astrometry(models_compare, all_sources=True, dir=directory.value,
                                           label=['clean', 'ddfacet', 'lwimager', 'tclean', 'wsclean'])

## Morphological Measurements

In [ ]:
shape = aimfast.aimfast.plot_morphology(models_compare, all_sources=True, dir=directory.value,
                                        label=['clean', 'ddfacet', 'lwimager', 'tclean', 'wsclean'])

## Spectral Index Measurements

In [ ]:
spectrum = aimfast.aimfast.plot_spectrum(models_compare, all_sources=True, dir=directory.value,
                                         label=['clean', 'ddfacet', 'lwimager', 'tclean', 'wsclean'])

In [ ]:
import matplotlib
import pylab as plt
from scipy.stats import norm
fig = plt.figure()
spi_bins = num_bins = 5
i = -1
j = 0
t = -1
counter = 0
flux_im = dict()
I_min_max = []
err_I_min_max = []

global_table_data = [["Global", "Mean", "Median", "Mode", "Standard Deviation"]]
label=['clean', 'ddfacet', 'lwimager', 'tclean', 'wsclean']

MEAN = {}
MEDIAN = {}
MODE = {}
STD = {}
FLUX_RANGE = {}

for input_model, output_model in sorted(models_compare.items()):
    i += 1
    t += 1
    bin_table_data = [[sorted(label)[t].upper(), "Mean", "Median", "Mode", "Standard Deviation", "Flux range (mJy)"]]
    header = '{}-model_1'.format(label[t])
    counter+=1
    I_in = []
    name_labels = []
    spi_in_data = []
    spi_out_data = []
    spi_err_data = []
    dist_from_phase = []
    key = 'spectrum'
    num_data_points = len(spectrum[header][key])
    for n in range(num_data_points):
        spi_out_data.append(spectrum[header][key][n][0])
        spi_err_data.append(spectrum[header][key][n][1])
        spi_in_data.append(spectrum[header][key][n][2])
        dist_from_phase.append(spectrum[header][key][n][3])
        I_in.append(spectrum[header][key][n][4])
        name_labels.append(spectrum[header][key][n][5])
    zipped_props = zip(I_in, spi_out_data, spi_err_data, spi_in_data, dist_from_phase, name_labels)
    (I_in, spi_out_data, spi_err_data, spi_in_data, dist_from_phase, name_labels) = zip(
        *sorted(zipped_props, key=lambda x: x[0]))
    
#===========================================================================================================
    spi_in_data_stats = []
    spi_out_data_stats = []
    for spi_in, spi_out in zip(spi_in_data, spi_out_data):
        if spi_out:
            spi_in_data_stats.append(spi_in)
            spi_out_data_stats.append(spi_out)
    spi_R_score = r2_score(spi_in_data_stats, spi_out_data_stats)
    spi_MSE = mean_squared_error(spi_in_data_stats, spi_out_data_stats)
    spi_out_in = [float(spi_out)/spi_in for spi_out,spi_in in
                      zip(spi_out_data_stats,spi_in_data_stats)]
    spi_in_data = spi_in_data_stats
    spi_out_data = spi_out_data_stats
#===========================================================================================================
    
    ranger = num_data_points/num_bins
    start, end = [-ranger, 0]
    if num_bins > 1:
        meanss = {}
        modess = {}
        medianss = {}
        stdss = {}
        flux_rangess = {}
        for b in range(num_bins):
            end += ranger
            start += ranger

            low = sorted(np.array(I_in)[start:num_data_points
                    if (b + 1) == num_bins
                    else end]*aimfast.aimfast.FLUX_UNIT_SCALER['milli'][0])[0]
            high = sorted(np.array(I_in)[start:num_data_points
                    if (b + 1) == num_bins
                    else end]*aimfast.aimfast.FLUX_UNIT_SCALER['milli'][0])[-1]
            I_range = "{}->{} {}".format(low, high, aimfast.aimfast.FLUX_UNIT_SCALER['milli'][1])
            
            spi_out_data_range = spi_out_data[start:num_data_points if (b + 1) == num_bins else end],
            n, bins, patches = matplotlib.pyplot.hist(spi_out_data_range,
                                                      spi_bins, normed=False, facecolor='blue',
                                                      orientation='horizontal', alpha=0.5)
            (mu, sigma) = norm.fit(spi_out_data_range)
            median = np.median(spi_out_data_range)
            peak = [bins[i] for i,v in enumerate(n) if v == n.max()][0]
            meanss[b] = mu
            modess[b] = peak
            medianss[b] = median
            stdss[b] = sigma
            flux_rangess[b] = I_range
            
            bin_table_data.append(["BIN-{:d}".format(b+1), "{:4f}".format(mu), "{:4f}".format(median),
                                  "{:4f}".format(peak), "{:4f}".format(sigma), "{:s}".format(I_range)])
            
        MEAN[label[t]] = meanss
        MEDIAN[label[t]]  = medianss
        MODE[label[t]] = modess
        STD[label[t]] = stdss
        FLUX_RANGE[label[t]] = flux_rangess
            
    print(to_text(bin_table_data))
    n, bins, patches = matplotlib.pyplot.hist(spi_out_data_range,
                                              spi_bins, normed=False, facecolor='blue',
                                              orientation='horizontal', alpha=0.5)
    (mu, sigma) = norm.fit(spi_out_data)
    median = np.median(spi_out_data)
    peak = [bins[i] for i,v in enumerate(n) if v == n.max()][0]
    global_table_data.append([header.split('-model_1')[0].upper(), "{:4f}".format(mu), "{:4f}".format(median),
                            "{:4f}".format(peak), "{:4f}".format(sigma)])
    
    MEAN[label[t]]['g'] = mu
    MEDIAN[label[t]]['g']  = median
    MODE[label[t]]['g'] = peak
    STD[label[t]]['g'] = sigma
    
    counter += 1

#overleaf = """
#    & & BIN-{} [{}] & \\
# \hline\hline
# Mean & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}\\
# \hline
# Median & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}\\
# \hline
# Mode & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}\\
# \hline
# Standard deviation & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}\\
# \hline\hline
# """
#overall = """
#     & & & Overall & & \\
#\hline\hline
# Mean & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}\\
# \hline
# Median & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}\\
# \hline
# Mode & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}\\
# \hline
# Standard deviation & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}\\ [1ex]
#"""

#for b in range(num_bins):
#    print(overleaf.format(b+1, FLUX_RANGE['clean'][b],
#                           MEAN['clean'][b], MEAN['ddfacet'][b], MEAN['lwimager'][b], MEAN['tclean'][b], MEAN['wsclean'][b],
#                           MEDIAN['clean'][b], MEDIAN['ddfacet'][b], MEDIAN['lwimager'][b], MEDIAN['tclean'][b], MEDIAN['wsclean'][b],
#                           MODE['clean'][b], MODE['ddfacet'][b], MODE['lwimager'][b], MODE['tclean'][b], MODE['wsclean'][b],
#                           STD['clean'][b], STD['ddfacet'][b], STD['lwimager'][b], STD['tclean'][b], STD['wsclean'][b]
#                           ))
#b = 'g'
#print(overall.format(
#                           MEAN['clean'][b], MEAN['ddfacet'][b], MEAN['lwimager'][b], MEAN['tclean'][b], MEAN['wsclean'][b],
#                           MEDIAN['clean'][b], MEDIAN['ddfacet'][b], MEDIAN['lwimager'][b], MEDIAN['tclean'][b], MEDIAN['wsclean'][b],
#                           MODE['clean'][b], MODE['ddfacet'][b], MODE['lwimager'][b], MODE['tclean'][b], MODE['wsclean'][b],
#                           STD['clean'][b], STD['ddfacet'][b], STD['lwimager'][b], STD['tclean'][b], STD['wsclean'][b]
#                           ))
#    

                               
plt.close(fig)
print(to_text(global_table_data))

## Dynamic Range

In [ ]:
prefix = 'meerkat_{}_'.format(sky)
if sky in ['point']:
    plot_title = 'Point Source Simulation'
    expected_dr = np.array([100, 100, 100, 100, 100])
elif sky in ['extended']:
    plot_title = 'Extended Source Simulation'
    expected_dr = np.array([67.5, 67.5, 67.5, 67.5, 67.5])
elif sky in ['hdr']:
    plot_title = 'High SNR Source Simulation'
    expected_dr = np.array([10000, 10000, 10000, 10000, 10000])
beam_factor = 6
DRs = OrderedDict()
im_titles = []
dir = directory.value
for m, r in models_dr.items():
    DRs[r.split('.')[0].split(prefix)[-1].split('_')[0].upper()] = model_dynamic_range(
        '{:s}/{:s}'.format(dir, m),'{:s}/{:s}'.format(dir, r), 6, beam_factor)
zipped_props = zip(DRs.keys(), DRs.values())
ims, values = zip(*sorted(zipped_props, key=lambda x: x[0]))
drs, names = [], []
for val in values:
    drs.append(val['global_rms'])
    names.append("DR={:f}".format(val['global_rms']))
aimfast.aimfast.aimfast_plotly(X1=ims, Y1=np.array(drs), plot_title=plot_title,
                               X2=ims, Y2=expected_dr,
                               x_title='Images', y_title='Dynamic Range')

## Residual Statistics

In [ ]:
aimfast.aimfast.print_residual_stats(sorted(RESIDUALS.keys()), prefix='meerkat_{}_'.format(sky),
                                     suffix='residual.fits', replace='cube-residual',
                                     normality='normaltest', units='mJy', dir=directory.value)

#### Source residual-to-noise ration measurements

In [ ]:
aimfast.aimfast.plot_residuals_noise(RESIDUALS,
                                     area_factor=6.0, dir=directory.value,
                                     skymodel='{}_skymodel1.txt'.format(sky),
                                     label=['clean', 'ddfacet', 'lwimager', 'tclean', 'wsclean'])

#### Random source residual-to-noise ration measurements

In [ ]:
aimfast.aimfast.plot_residuals_noise(RESIDUALS, skymodel=None,
                                     area_factor=5.0, points=100, dir=directory.value,
                                     label=['clean', 'ddfacet', 'lwimager', 'tclean', 'wsclean'])

## View Reconstructed Maps

In [ ]:
%%html
<iframe src="http://js9.si.edu" width=990 height=740></iframe>

## Source meta-data

In [ ]:
imagers = ['clean', 'ddfacet', 'lwimager', 'tclean', 'wsclean']
cb_cont = widgets.HBox(background_color='red',height='100px',width='100%')
Im = []
for imager in sorted(imagers):
    Im.append(widgets.Checkbox(description=imager,value=True,width=50))
cb_cont.children=[i for i in Im]

src_nm = widgets.Text(
    value='',
    placeholder='ay040',
    description='Source Name:',
    disabled=False
)
display(cb_cont)
display(src_nm)

In [ ]:
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="Search")
button.on_click(run_all)
display(button)

In [ ]:
im_search = dict()
for i, im in enumerate(sorted(imagers)):
    im_search[im] = 1 if cb_cont.children[i].value else 0
src_target = src_nm.value
results = spectrum
for k,v in sorted(im_search.items()):
    if v:
        model = '{}-model_1'.format(k)
        for i, src in enumerate(results[model]['flux']):
            if src_target == src[-1]:
                pos = results[model]['position'][i]
                print('============')
                print ("Source info:\n============\n"
                       "name = %s\n"
                       "In_flux = %s Jy\n"
                       "Out_flux = %s Jy\n"
                       "Out_flux_error = %s Jy\n"
                       "Model file = %s"
                       % (src[-1],
                          round(src[2], 6),
                          round(src[0], 6),
                          round(src[1], 6),
                          results[model]['models'][-1]))
                print('\n')